# Example to fetch SAR Sentinel-1 WV roughness figures from X-waves application

In [ ]:
import apixwaves

## define space and time query

In [ ]:
from apixwaves import timeSpaceFileListQuery
bbox = "30,-30,-30,10"
satellite = 'S1A'
#latmin,lonmin,latmax,lonmax =  30,-30,-30,10
date_interval = "20140101010101,20170106112608"
dictQuery = timeSpaceFileListQuery.rawQuery(bbox=bbox,satellite=satellite,date_interval=date_interval)
print(len(dictQuery))
print(dictQuery[0])

## display a single spectra

In [ ]:
url_image = 'https://xwaves-services.ifremer.fr/datavore/exp/datavore/public/S1B/S1B_WV_SLC__1SSV_20211221T165539_20211221T172051_030126_0398ED_658A.SAFE/s1b-wv1-slc-vv-20211221t170920-20211221t170923-030126-0398ed-057.tiff/S1B_WV_OCN__2SSV_20211221T165539_20211221T172051_030126_0398ED_D0BA.SAFE/s1b-wv1-ocn-vv-20211221t170920-20211221t170923-030126-0398ed-057.nc/realPart/20211221170920/imageSentinel/'
from matplotlib import pyplot as plt
#import fsspec
from PIL import Image
import requests
from io import BytesIO

response = requests.get(url_image)
img = Image.open(BytesIO(response.content))
img

## reconstruct path for images

In [ ]:
pattern_url_image = 'https://xwaves-services.ifremer.fr/datavore/exp/datavore/public/%s/%s/%s/%s/%s/imageSentinel/'
import os
import numpy as np
empty_field = np.ones((142),dtype=str)
empty_field[:] = ' '
empty_field = ''.join(empty_field)
imagetype = 'realPart' # or realPart or ..
imagetype = 'roughness'
cpt_ok = 0
for acq in dictQuery:
    #print(acq)
    sat = acq[3]
    tiffid = acq[10]
    if not np.array_equal(tiffid,empty_field):
        ocnuid = acq[11]
        startdate = acq[0]
        real_pat = os.path.join(pattern_url_image%(sat,tiffid,ocnuid,imagetype,startdate))
        print(real_pat)
        cpt_ok += 1
        response = requests.get(real_pat)
        img = Image.open(BytesIO(response.content))
        plt.figure()
        plt.imshow(img)
        if cpt_ok>3:
            
            break
